In [ ]:
from utils import show, getQuantizedColor
# import all in utils 
import utils as ut
from skimage import feature, morphology
import numpy as np
import cv2
import os
import numpy as np
from skimage import morphology
from skimage.morphology import medial_axis
from skimage.util import invert
from skimage.io import imread
%load_ext autoreload
%autoreload 2


In [ ]:
imgName = 'map1024.png'
projectName = 'map1024'
texturePath = '../../textures/map1024/'
outputPath = './outputs/'
outputPath = os.path.join(outputPath, projectName)

imgPath = os.path.join(texturePath, imgName)

image = imread(imgPath, as_gray=True)

height, width = image.shape
imgSize = (400, 400)
# imgSize = (width, height)
image = cv2.resize(image, imgSize)

if not os.path.exists(outputPath):
    os.makedirs(outputPath)

# Copy imgPath as input.png
inputPath = os.path.join(outputPath, 'input.png')
cv2.imwrite(inputPath, cv2.cvtColor(imread(imgPath), cv2.COLOR_BGR2RGB))

In [ ]:
image = cv2.imread(imgPath)
image = cv2.resize(image, imgSize)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
show(image_rgb)

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
show(gray)

In [ ]:
grayThreshold = 96
_, thresholded = cv2.threshold(gray, grayThreshold, 255, cv2.THRESH_BINARY)
show(thresholded)

In [ ]:
# CANNY
sigma = 4
_, thresholded = cv2.threshold(gray, grayThreshold, 255, cv2.THRESH_BINARY)
canny = feature.canny(thresholded, sigma=sigma)
show(canny)

In [ ]:
# Skeleton
sensitivity = 0.36
image = imread(imgPath, as_gray=True)
image = cv2.resize(image, imgSize)
binary_image = image > sensitivity
binary_image = invert(binary_image)

skeleton, distance = medial_axis(binary_image, return_distance=True)
skeleton = invert(skeleton)
show(skeleton)
cv2.imwrite(os.path.join(outputPath, 'skeleton.png'), (skeleton * 255).astype(np.uint8))

In [ ]:
image = cv2.imread(imgPath, 0)
image = cv2.resize(image, imgSize)

_, thresh = cv2.threshold(image, grayThreshold, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

blank_image = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
blank_image[:] = (255, 255, 255)
print(len(contours))

for i, contour in enumerate(contours):
    color = (np.random.randint(0, 255), np.random.randint(
        0, 255), np.random.randint(0, 255))

    cv2.drawContours(blank_image, contours, i, color, -1)


show(blank_image)


In [ ]:
figure = skeleton
skeleton_uint8 = (figure * 255).astype(np.uint8)
image = cv2.imread(imgPath, 0)
image = cv2.resize(image, imgSize)

_, thresh = cv2.threshold(image, grayThreshold, 255, cv2.THRESH_BINARY)

contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))

filled_image = cv2.cvtColor(skeleton_uint8, cv2.COLOR_GRAY2BGR)

points = []
for i, contour in enumerate(contours):
    color = (np.random.randint(0, 255), np.random.randint(
        0, 255), np.random.randint(0, 255))

    cv2.drawContours(filled_image, contours, i, color, -1)
    points.append(contour[0])

show(filled_image)


In [ ]:
copy = cv2.cvtColor(invert(skeleton_uint8), cv2.COLOR_GRAY2BGR)

for point in points:
    x, y = point[0]
    color = tuple(filled_image[y, x])
    color = (int(color[0]), int(color[1]), int(color[2]))
    cv2.floodFill(copy, None, (x, y), color)

show(copy)

In [ ]:
copy = cv2.cvtColor(invert(skeleton_uint8), cv2.COLOR_GRAY2BGR)
show(copy)
i = 0

provinceDataViews = []

for point in points:
    x, y = point[0]
    color = getQuantizedColor(i)
    provinceDataView = ut.ProvinceDataView(ut.generateRandomName(
    ), ut.getHexRandomColor(i), i, ut.getQuantizedValue(color), {"x": int(x), "y": int(y)})
    provinceDataViews.append(provinceDataView)
    cv2.floodFill(copy, None, (x, y), color)
    i += 1

show(copy)

In [ ]:
import json

my_objects = []
my_objects.append(ut.ProvinceDataView("Lavender", "#F6A2AC",
                  25, 215, {"x": 1209.8106, "y": 224.7444}))
my_objects.append(ut.ProvinceDataView("Rose", "#FFC0CB",
                  31, 198, {"x": 987.6543, "y": 321.0987}))
my_objects.append(ut.ProvinceDataView("Sunflower", "#FFCE00",
                  42, 123, {"x": 543.2109, "y": 876.5432}))

# Take 3 first elements of provinceDataViews and put them in a new array
# provinceDataViews = my_objects[:2]
print(my_objects[0].name, my_objects[0].color, my_objects[0].id,
      my_objects[0].hash, my_objects[0].fixed_position)
print(provinceDataViews[0].name, provinceDataViews[0].color, provinceDataViews[0].id,
      provinceDataViews[0].hash, provinceDataViews[0].fixed_position)
# Again but with types
print(type(my_objects[0].name), type(my_objects[0].color), type(
    my_objects[0].id), type(my_objects[0].hash), type(my_objects[0].fixed_position))
print(type(provinceDataViews[0].name), type(provinceDataViews[0].color), type(
    provinceDataViews[0].id), type(provinceDataViews[0].hash), type(provinceDataViews[0].fixed_position))


provinceDataViewsJson = ut.convertProvinceDataViewArrayToJson(
    provinceDataViews)
json_data = json.dumps(provinceDataViewsJson,   indent=4)


jsonPath = os.path.join(outputPath, projectName + '.json')
with open(jsonPath, 'w') as file:
    file.write(json_data)